# Statistical models in R
This notebook is covering:
1. VECM

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go

In [2]:
# rpy2 is a Python package that allows you to run R code from Python
%pip install rpy2

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Load the rpy2 extension to use R in Jupyter
%load_ext rpy2.ipython

The magic function `%%R` is used for running R code in Jupyter

In [4]:
%%R
source("vecm.r")
hub1_name <- "ttf"
hub2_name <- "the"
validation_size <- 250
test_size <- 250
window_size <- 5

Loading required package: tsDyn
Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 
Loading required package: Metrics

Attaching package: ‘Metrics’

The following object is masked from ‘package:tsDyn’:

    mse

Loading required package: dplyr

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [5]:
%%R
pairs <- list(c("ttf", "the"), 
            c("ttf", "nbp"), 
            c("the", "nbp")
            )
window_sizes <- c(5, 10)
for (i in 1:length(pairs)) {
    for(k in 1:length(window_sizes)) {
        pair <- pairs[[i]]
        hub1_name <- pair[1]
        hub2_name <- pair[2]
        window_size <- window_sizes[k]
        vecm_system(hub1_name, hub2_name, validation_size, test_size, window_size, verbose=FALSE, save=TRUE)
        print(paste("Finished", hub1_name, hub2_name, window_size))
    }
}

[1] "Finished ttf the 5"
[1] "Finished ttf the 10"
[1] "Finished ttf nbp 5"
[1] "Finished ttf nbp 10"
[1] "Finished the nbp 5"
[1] "Finished the nbp 10"


In [6]:
%%R
vecm_output <- vecm_system("the", "nbp", validation_size, test_size, 5, verbose=TRUE, save=TRUE)

[1] "Selected number of lags: 5"
[1] "Pair: the | nbp"
[1] "Window Size: 5"
[1] "Test Size: 250"
[1] "the: Mean Absolute Error: 2.071"
[1] "the: Root Mean Squared Error: 2.927"
[1] "nbp: Mean Absolute Error: 2.137"
[1] "nbp: Root Mean Squared Error: 3.034"


In [7]:
%%R
training_model <- vecm_training_model(hub1_name, hub2_name, test_size, window_size)
summary(training_model)

#############
###Model VECM 
#############
Full sample size: 1285 	End sample size: 1279
Number of variables: 2 	Number of estimated slope parameters 24
AIC 6868.471 	BIC 6997.317 	SSR 87041.64
Cointegrating vector (estimated by ML):
   hub1     hub2
r1    1 -1.38433


              ECT               Intercept          hub1 -1          
Equation hub1 0.0220(0.0118).   0.2362(0.2023)     0.2716(0.0663)***
Equation hub2 0.0414(0.0122)*** 0.4365(0.2105)*    0.3090(0.0690)***
              hub2 -1             hub1 -2            hub2 -2           
Equation hub1 -0.1052(0.0641)     -0.1819(0.0661)**  0.0513(0.0638)    
Equation hub2 -0.1695(0.0667)*    -0.2080(0.0687)**  0.0747(0.0664)    
              hub1 -3            hub2 -3           hub1 -4          
Equation hub1 -0.2619(0.0661)*** 0.2494(0.0637)*** 0.2035(0.0665)** 
Equation hub2 -0.2687(0.0688)*** 0.2397(0.0663)*** 0.2438(0.0692)***
              hub2 -4             hub1 -5            hub2 -5          
Equation hub1 -0.0484(0.0641)

In [8]:
%%R
hub1_predictions <- vecm_output$predictions$hub1
hub1_actuals <- vecm_output$actuals$hub1
hub2_predictions <- vecm_output$predictions$hub2
hub2_actuals <- vecm_output$actuals$hub2


In [9]:
%%R
hub1_forecast_comparison <- data.frame(
  Actual = hub1_actuals,
  Forecasted = hub1_predictions
)

In [10]:
%R -o hub1_forecast_comparison
%R -o hub1_name

hub1_forecast_comparison = hub1_forecast_comparison

actual_trace = go.Scatter(
    x=hub1_forecast_comparison.index,
    y=hub1_forecast_comparison['Actual'],
    mode='lines',
    name='Actual Prices'
)

forecasted_trace = go.Scatter(
    x=hub1_forecast_comparison.index,
    y=hub1_forecast_comparison['Forecasted'],
    mode='lines',
    name='Forecasted Prices'
)

# Create the plot
layout = go.Layout(
    title= hub1_name.item() + ': VECM Actual vs Forecasted Prices',
    xaxis=dict(title='Index'),
    yaxis=dict(title='Price'),
)

fig = go.Figure(data=[actual_trace, forecasted_trace], layout=layout)
fig.show()


In [11]:
%%R
hub2_forecast_comparison <- data.frame(
  Actual = hub2_actuals,
  Forecasted = hub2_predictions
)

In [12]:
%R -o hub2_forecast_comparison
%R -o hub2_name

hub2_forecast_comparison = hub2_forecast_comparison

actual_trace = go.Scatter(
    x=hub2_forecast_comparison.index,
    y=hub2_forecast_comparison['Actual'],
    mode='lines',
    name='Actual Prices'
)

forecasted_trace = go.Scatter(
    x=hub2_forecast_comparison.index,
    y=hub2_forecast_comparison['Forecasted'],
    mode='lines',
    name='Forecasted Prices'
)

# Create the plot
layout = go.Layout(
    title= hub2_name.item() + ': VECM Actual vs Forecasted Prices',
    xaxis=dict(title='Index'),
    yaxis=dict(title='Price'),
)

fig = go.Figure(data=[actual_trace, forecasted_trace], layout=layout)
fig.show()
